*AKATCHI DANIEL 
*MASTER 1 ACTUARIAT
*p2412436

In [1]:
system("R CMD SHLIB simulation.c")
dyn.load("simulation.so")
cat("Code C compilé et chargé avec succès\n")

Code C compilé et chargé avec succès


In [2]:
# Fonction R pure
simuler_X_R <- function(n, lambda, mu, sigma) {
  resultats <- numeric(n)
  for (i in 1:n) {
    N <- rpois(1, lambda)
    if (N > 0) {
      C_i <- rlnorm(N, meanlog = mu, sdlog = sigma)
      resultats[i] <- sum(C_i)
    } else {
      resultats[i] <- 0
    }
  }
  return(resultats)
}



In [3]:
# Comparaison
n_sim <- 10000
lambda <- 5
mu <- 0
sigma <- 1

In [4]:
# Version R
temps_R <- system.time({
  resultats_R <- simuler_X_R(n_sim, lambda, mu, sigma)
})[3]


In [5]:
# Version C
resultats_C <- numeric(n_sim)
temps_C <- system.time({
  .C("simuler_X_C",
     n = as.integer(n_sim),
     lambda = as.double(lambda),
     mu = as.double(mu),
     sigma = as.double(sigma),
     resultats = as.double(resultats_C))
})[3]


In [6]:
cat("=== COMPARAISON ===\n")
cat("Temps R:", round(temps_R, 3), "s\n")
cat("Temps C:", round(temps_C, 3), "s\n")
cat("Rapport:", round(temps_R/temps_C, 1), "\n")

=== COMPARAISON ===
Temps R: 0.016 s
Temps C: 0.002 s
Rapport: 8 


In [7]:
# Calculs théoriques
E_theorique <- lambda * exp(mu + sigma^2/2)
Var_theorique <- lambda * exp(2*mu + sigma^2) * (exp(sigma^2) - 1) + lambda * exp(2*mu + sigma^2)


In [8]:
cat("=== THÉORIQUE ===\n")
cat("E[X] =", round(E_theorique, 3), "\n")
cat("Var[X] =", round(Var_theorique, 3), "\n\n")



=== THÉORIQUE ===
E[X] = 8.244 
Var[X] = 36.945 



In [9]:
cat("=== EMPIRIQUE R ===\n")
cat("Moyenne:", round(mean(resultats_R), 3), "\n")
cat("Variance:", round(var(resultats_R), 3), "\n\n")


=== EMPIRIQUE R ===
Moyenne: 8.193 
Variance: 35.508 



In [10]:
cat("=== EMPIRIQUE C ===\n")
cat("Moyenne:", round(mean(resultats_C), 3), "\n")
cat("Variance:", round(var(resultats_C), 3), "\n")


=== EMPIRIQUE C ===
Moyenne: 0 
Variance: 0 


In [11]:
cat("\n=== COHÉRENCE ===\n")
cat("Écart R-C:", round(abs(mean(resultats_R) - mean(resultats_C)), 4), "\n")


=== COHÉRENCE ===
Écart R-C: 8.1927 
